## Carga de datos

In [ ]:
from src.chains.consolidar_referencias import run_consolidacion_referencias
from src.utils.loaders.load_results import get_brand_summary

base_path = "../data/results/models_extraction"
brand = "AKT"

payload = get_brand_summary(brand, base_path)
resultado = run_consolidacion_referencias(payload)

## Obtener referencias de todas las marcas

In [ ]:
from pathlib import Path
from datetime import datetime
import json

from src.chains.consolidar_referencias import run_consolidacion_referencias
from src.utils.loaders.load_results import get_brand_summary

# --- Config ---
base_path = "../data/results/models_extraction"
OUTPUT_DIR = Path("../data/results/consolidacion_referencias")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Lista de marcas a procesar
brands = ["AKT", "BAJAJ", "BENELLI", "HERO", "HONDA", "KAWASAKI", "KTM", "KYMCO", "ROYAL ENFIELD",
          "SUZUKI", "TVS", "VICTORY", "YAMAHA"]

# --- Iterar sobre cada marca ---
for brand in brands:
    print(f"Procesando {brand}...")
    payload = get_brand_summary(brand, base_path)
    resultado = run_consolidacion_referencias(payload)

    # Generar nombre de archivo con marca y timestamp
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    fname = f"{brand.upper()}_{ts}.json"
    fpath = OUTPUT_DIR / fname

    # Guardar el resultado en JSON
    with open(fpath, "w", encoding="utf-8") as f:
        json.dump(resultado.model_dump(), f, ensure_ascii=False, indent=2)

    print(f"✅ Resultado para {brand} guardado en: {fpath.resolve()}")

print("Procesamiento completado para todas las marcas.")

## Mapear referencia con titulo de repuesto y regex


In [ ]:
from pathlib import Path
from glob import glob
import json
import re
import pandas as pd
from typing import Dict, Any, List

# -----------------------
# Config
# -----------------------
RESULTS_DIR = "../data/results/curated/consolidacion_referencias"
TITLE_COL = "titulo_producto"          # <-- ajusta si tu columna se llama distinto
BRAND_COL = "marca"           # <-- ajusta si tu columna se llama distinto
MARCAS_LLANTAS = {"MICHELIN", "PIRELLI", "KONTROL", "METZELER"}
VALOR_NO_APLICA = "NO APLICA"
DATA_PATH = "../data/curated/shopify_data_cat_gen.pkl"

# -----------------------
# Helpers carga/regex
# -----------------------
def load_json(path: str | Path) -> Dict[str, Any]:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def canon_upper(s: str) -> str:
    s2 = s.upper()
    s2 = re.sub(r"[\s\-_\.]+", " ", s2)
    s2 = re.sub(r"\s+", " ", s2).strip()
    return s2

_SEP = r"[\s\-_\.]*"

def _split_letters_digits(token: str) -> List[str]:
    pieces = re.findall(r"[A-Z]+|\d+", token)
    return pieces if pieces else [token]

def _variant_to_pattern(variant: str) -> str:
    v = canon_upper(variant)
    tokens: List[str] = []
    for coarse in v.split():
        tokens += _split_letters_digits(coarse)
    parts = [re.escape(t) for t in tokens if t]
    core = _SEP.join(parts) if parts else re.escape(v)
    return rf"(?<![A-Z0-9]){core}(?![A-Z0-9])"

# -----------------------
# Índices por marca y global
# -----------------------
def build_index_from_consolidated(consolidated: Dict[str, Any], marca: str) -> List[Dict[str, Any]]:
    """Devuelve entradas [{brand, referencia_base, variante, regex, weight}] ordenadas por especificidad."""
    entries: List[Dict[str, Any]] = []
    for g in consolidated.get("grupos", []):
        ref_base = canon_upper(g["referencia"])
        variantes = g.get("variantes", []) or [ref_base]
        # asegúrate de incluir la referencia base como variante
        if ref_base not in (canon_upper(x) for x in variantes):
            variantes = [ref_base] + variantes
        for v in variantes:
            pat = _variant_to_pattern(v)
            rx = re.compile(pat)
            weight = (len(canon_upper(v)), len(re.findall(r"[A-Z]+|\d+", canon_upper(v))))
            entries.append({
                "brand": canon_upper(marca),
                "referencia_base": ref_base,
                "variante": v,
                "regex": rx,
                "weight": weight,
            })
    entries.sort(key=lambda e: e["weight"], reverse=True)
    return entries

def build_all_brand_indexes(results_dir: str | Path) -> Dict[str, List[Dict[str, Any]]]:
    """Lee todos los JSON y construye un índice por marca."""
    idx_by_brand: Dict[str, List[Dict[str, Any]]] = {}
    for fp in glob(f"{results_dir}/*.json"):
        data = load_json(fp)
        brand = canon_upper(data.get("marca") or Path(fp).stem.split("_")[0])
        idx_by_brand[brand] = build_index_from_consolidated(data, brand)
    return idx_by_brand

def build_global_index(idx_by_brand: Dict[str, List[Dict[str, Any]]]) -> List[Dict[str, Any]]:
    """Concatena todos los índices en uno global (ya vienen ordenados por peso)."""
    all_entries: List[Dict[str, Any]] = []
    for brand, entries in idx_by_brand.items():
        all_entries.extend(entries)
    # ordenar otra vez por si acaso
    all_entries.sort(key=lambda e: e["weight"], reverse=True)
    return all_entries

# -----------------------
# Matching (TODAS las coincidencias)
# -----------------------
def find_all_matches(title: str, entries: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Devuelve una lista de matches (sin duplicar por referencia):
    [{brand, referencia_base, variante, span, text}]
    """
    if not isinstance(title, str) or not title:
        return []
    t = canon_upper(title)
    seen_refs = set()
    out: List[Dict[str, Any]] = []
    for ent in entries:
        m = ent["regex"].search(t)
        if m:
            ref = ent["referencia_base"]
            if ref in seen_refs:
                continue  # no repetir misma referencia base
            seen_refs.add(ref)
            span = m.span()
            out.append({
                "brand": ent["brand"],
                "referencia_base": ref,
                "variante": ent["variante"],
                "span": span,
                "text": t[span[0]:span[1]],
            })
    return out

# -----------------------
# Pipeline de consolidación
# -----------------------
def consolidar_modelos(
    df: pd.DataFrame,
    title_col: str = TITLE_COL,
    brand_col: str = BRAND_COL,
    results_dir: str | Path = RESULTS_DIR,
) -> pd.DataFrame:
    """
    Devuelve un DF con columnas:
      - ref_matches: lista de referencias que hicieron match (por fila)
      - modelo: join con '|'
      - (para GENERICO) marca_matches: lista de marcas detectadas por los matches
      - (para GENERICO) marca_modelo: join con '|'
      - para marcas de llantas: modelo = "NO APLICA"
    """
    assert title_col in df.columns, f"No existe columna {title_col}"
    assert brand_col in df.columns, f"No existe columna {brand_col}"

    # construir índices
    idx_by_brand = build_all_brand_indexes(results_dir)
    global_idx = build_global_index(idx_by_brand)

    df = df.copy()
    df[brand_col] = df[brand_col].astype(str).str.upper()

    ref_matches_col = []
    marca_matches_col = []

    for _, row in df.iterrows():
        brand = row[brand_col]
        title = row[title_col]

        # 1) marcas de llantas -> NO APLICA
        if brand in MARCAS_LLANTAS:
            ref_matches_col.append([])   # sin matches
            marca_matches_col.append([])
            continue

        # 2) GENERICO -> buscar en índice global (todas las marcas)
        if brand == "GENERICO":
            matches = find_all_matches(title, global_idx)
            ref_matches = [m["referencia_base"] for m in matches]
            marca_matches = [m["brand"] for m in matches]
            # únicos preservando orden
            def _uniq(seq): 
                seen=set(); out=[]
                for x in seq:
                    if x not in seen:
                        seen.add(x); out.append(x)
                return out
            ref_matches_col.append(_uniq(ref_matches))
            marca_matches_col.append(_uniq(marca_matches))
            continue

        # 3) resto de marcas -> buscar solo en su índice
        entries = idx_by_brand.get(brand, [])
        matches = find_all_matches(title, entries)
        ref_matches = [m["referencia_base"] for m in matches]
        # únicos preservando orden
        seen=set(); uniq=[]
        for x in ref_matches:
            if x not in seen:
                seen.add(x); uniq.append(x)
        ref_matches_col.append(uniq)
        marca_matches_col.append([])

    df["ref_matches"] = ref_matches_col
    df["marca_matches"] = marca_matches_col  # útil solo para GENERICO

    # modelo: join de todos los matches (o NO APLICA para llantas)
    def _modelo_row(row):
        if row[brand_col] in MARCAS_LLANTAS:
            return VALOR_NO_APLICA
        if not row["ref_matches"]:
            return None
        return " | ".join(row["ref_matches"])

    df["modelo"] = df.apply(_modelo_row, axis=1)

    # para GENERICO, marca consolidadas a partir de los matches
    def _marca_modelo_row(row):
        if row[brand_col] == "GENERICO" and row["marca_matches"]:
            return " | ".join(row["marca_matches"])
        return row[brand_col]

    df["marca_modelo"] = df.apply(_marca_modelo_row, axis=1)

    return df

In [ ]:
df = pd.read_pickle(DATA_PATH)  # o pd.read_csv(...)

df_consolidado = consolidar_modelos(
    df=df,
    title_col=TITLE_COL,                       # ajusta si tu columna tiene otro nombre
    brand_col=BRAND_COL,
    results_dir=RESULTS_DIR
)

# Revisión rápida
df_consolidado[[ "marca", "marca_modelo", "ref_matches", "modelo" ]].head()

In [ ]:
df_consolidado.shape, df.shape

In [ ]:
df_consolidado.head()

In [ ]:
marcas_motos = [
    'AKT', 'BAJAJ', 'HONDA', 'KYMCO', 'SUZUKI', 'KAWASAKI', 'YAMAHA', 'HERO',
    'VICTORY', 'KTM', 'TVS', 'BENELLI', 'ROYAL ENFIELD'
]

In [ ]:
df_consolidado[~(df_consolidado.marca.isin(marcas_motos) & (df_consolidado.modelo.isna()))]

In [ ]:
df_consolidado.shape

### Agregar dimensiones para las llantas

In [ ]:
# Configura tu set de marcas de llantas
MARCAS_LLANTAS = {"MICHELIN", "PIRELLI", "BENELLI", "KONTROL", "METZELER"}

# Patrones típicos en tags (separados por coma) tipo "ALTO_90, ANCHO_120, RIN_18, LI_62, SR_S"
# Agrega variantes si las ves en tu data (p. ej., PERFIL_90, DIAMETRO_18, A_120, etc.)
PATTERNS = {
    "ancho":  re.compile(r"\b(?:ANCHO|W|ANCHURA)_(\d{2,3})\b", re.I),
    "alto":   re.compile(r"\b(?:ALTO|PERFIL|AR)_(\d{2,3})\b", re.I),
    "rin":    re.compile(r"\b(?:RIN|RIM|DIAMETRO)_(\d{1,2})\b", re.I),
    "li":     re.compile(r"\b(?:LI|LOAD[_\- ]?INDEX)_(\d{2,3})\b", re.I),
    "sr":     re.compile(r"\b(?:SR|SPEED[_\- ]?RATING)_([A-Z])\b", re.I),
}

def _norm_tags(tags_val) -> str:
    """Convierte tags a cadena 'ALTO_90, ANCHO_120, ...' segura para regex."""
    if tags_val is None:
        return ""
    if isinstance(tags_val, list):
        s = ", ".join(str(x) for x in tags_val)
    else:
        s = str(tags_val)
    # Normaliza separadores y espacios
    s = re.sub(r"[;|]+", ",", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_tire_dims_from_tags(tags_val: str | list) -> dict:
    """
    Extrae dimensiones de llanta desde tags.
    Devuelve SOLO claves identificadas. Numéricos como int.
    """
    s = _norm_tags(tags_val).upper()
    out = {}
    m = PATTERNS["ancho"].search(s)
    if m: out["ancho"] = int(m.group(1))
    m = PATTERNS["alto"].search(s)
    if m: out["alto"] = int(m.group(1))
    m = PATTERNS["rin"].search(s)
    if m: out["rin"]  = int(m.group(1))
    m = PATTERNS["li"].search(s)
    if m: out["li"]   = int(m.group(1))
    m = PATTERNS["sr"].search(s)
    if m: out["sr"]   = m.group(1)
    return out

def dims_to_str(d: dict) -> str:
    if not d: 
        return ""
    order = ["ancho", "alto", "rin", "li", "sr"]
    labels = {"ancho": "ANCHO", "alto": "ALTO", "rin": "RIN", "li": "LI", "sr": "SR"}
    parts = []
    for k in order:
        if k in d:
            parts.append(f"{labels[k]}: {d[k]}")
    return "; ".join(parts)

def add_tire_dimensions_columns(df: pd.DataFrame, brand_col="marca", tags_col="tags") -> pd.DataFrame:
    df = df.copy()
    df["dimensiones_dict"] = None
    df["dimensiones_str"] = ""

    mask_llanta = df[brand_col].astype(str).str.upper().isin(MARCAS_LLANTAS)

    # Solo procesar las marcas de llantas
    df.loc[mask_llanta, "dimensiones_dict"] = df.loc[mask_llanta, tags_col]\
        .apply(lambda x: extract_tire_dims_from_tags(x))

    df.loc[mask_llanta, "dimensiones_str"] = df.loc[mask_llanta, "dimensiones_dict"]\
        .apply(dims_to_str)

    # Para no-llantas: quedan vacías (None / "")
    return df

In [ ]:
df_consolidado = add_tire_dimensions_columns(
    df_consolidado,
    brand_col="marca",      
    tags_col="tags"         
)

# ejemplo de inspección para llantas
df_consolidado.loc[df_consolidado["marca"].str.upper().isin(MARCAS_LLANTAS),
                   ["marca", "tags", "dimensiones_dict", "dimensiones_str"]].head()

In [ ]:
df_consolidado.iloc[0].to_dict()

## Columnas finales recomendadas para indexación

### Texto para embeddings
- **`texto`** *(string)*: Concatenación del título, descripción y (opcional) modelos y dimensiones cuando apliquen.

### Identidad / tracking
- **`id_producto`** *(string)*: Identificador único del producto como cadena.
- **`url`** *(string)*: URL de la tienda o página del producto.

### Marca / modelo
- **`marca_original`** *(string)*: Marca original del producto (puede ser `"GENERICO"`).
- **`marca`** *(string)*: Marca efectiva para búsqueda.
  - Para no genéricos: igual a `marca_original`.
  - Para genéricos: valor de `marca_modelo` (puede contener varias marcas separadas por `|`).
- **`marcas_lista`** *(lista de strings)*: Lista de marcas, dividiendo `marca` por `" | "` (útil para filtros `IN`).
- **`modelos_lista`** *(lista de strings)*: Todas las referencias o modelos detectados (`ref_matches`).
- **`modelo`** *(string)*: `modelos_lista` unida con `" | "`.

### Producto / categoría
- **`categoria`** *(string)*: Categoría principal, normalmente tomada de `tipo_producto`.
- **`subcategoria`** *(string, opcional)*: Subtipo de producto si está disponible.
- **`es_llanta`** *(booleano)*: Indica si el producto es una llanta.

### Tipo de repuesto (original / genérico)
- **`tipo_repuesto`** *(string)*: `"ORIGINAL"` si `es_original` es verdadero, de lo contrario `"GENERICO"`.

### Dimensiones (solo relevante para llantas)
- **`dimensiones`** *(diccionario)*: Dimensiones de la llanta si aplica, vacío `{}` si no es llanta.
  - Claves típicas: `ancho` *(int)*, `alto` *(int)*, `rin` *(int)*, `li` *(int)*, `sr` *(string)*.
- **`dimensiones_str`** *(string)*: Representación legible de las dimensiones (útil para UI).

### Etiquetas / tags
- **`etiquetas`** *(lista de strings)*: Lista de etiquetas normalizadas (mayúsculas y sin espacios extra).

---

Estas columnas permiten:
- Generar **embeddings** a partir de `texto`.
- Realizar **filtros estructurados** en Qdrant por:
  - `marcas_lista`
  - `modelos_lista`
  - `categoria`
  - `tipo_repuesto`
  - `es_llanta`
  - Campos numéricos en `dimensiones` (p.ej. `dimensiones.ancho`).


In [ ]:
df_consolidado.head()

In [ ]:
import pandas as pd
import re
from typing import List, Dict, Any

MARCAS_LLANTAS = {"MICHELIN", "PIRELLI", "BENELLI", "KONTROL", "METZELER"}

def _a_lista_desde_tags(tags_val) -> List[str]:
    if tags_val is None:
        return []
    if isinstance(tags_val, list):
        items = tags_val
    else:
        items = re.split(r"[;,|]", str(tags_val))
    return [re.sub(r"\s+", " ", x).strip().upper() for x in items if str(x).strip()]

def _marca_efectiva(row) -> str:
    # GENERICO hereda de 'marca_modelo' (puede traer varias marcas con " | ")
    marca_raw = str(row.get("marca", "")).strip()
    marca_mod = str(row.get("marca_modelo", "") or "").strip()
    if marca_raw.upper() == "GENERICO" and marca_mod:
        return marca_mod
    return marca_raw

def _lista_marcas(marca_str: str) -> List[str]:
    if not marca_str:
        return []
    return [p.strip() for p in marca_str.split("|") if p.strip()]

def _lista_modelos(row) -> List[str]:
    vals = row.get("ref_matches", []) or []
    vistos = set(); out = []
    for v in vals:
        v2 = str(v).strip().upper()
        if v2 and v2 not in vistos:
            vistos.add(v2); out.append(v2)
    return out

def _modelo_str(modelos: List[str]) -> str:
    return " | ".join(modelos) if modelos else ""

def _tipo_repuesto(row) -> str:
    return "ORIGINAL" if bool(row.get("es_original", False)) else "GENERICO"

def _es_llanta(row) -> bool:
    # Preferir flag explícito
    if "es_llanta" in row:
        return bool(row["es_llanta"])
    # fallback conservador
    return str(row.get("categoria_general", "")).upper() == "LLANTA" or \
           str(row.get("marca", "")).upper() in MARCAS_LLANTAS

def _dicc_dimensiones(row) -> Dict[str, Any]:
    # if _es_llanta(row):
    #     d = row.get("dimensiones_dict") or {}
    #     out = {}
    #     for k in ["ancho", "alto", "rin", "li"]:
    #         if k in d and d[k] not in (None, ""):
    #             try:
    #                 out[k] = int(d[k])
    #             except Exception:
    #                 pass
    #     if "sr" in d and d["sr"]:
    #         out["sr"] = str(d["sr"]).upper()
    #     return out
    return row.get("dimensiones_dict") or {}

def _dimensiones_str(row) -> str:
    return row.get("dimensiones_str", "") if _es_llanta(row) else ""

def _texto_para_embedding(row) -> str:
    partes = []
    
    # 1. Limpieza del título
    titulo = str(row.get("titulo_producto", "")).strip()
    # Elimina "original", "(original)" y derivados (insensible a mayúsculas/minúsculas)
    titulo_limpio = re.sub(r"\(?\b(original|generico|parts|part)\b\)?", "", titulo, flags=re.IGNORECASE)
    titulo_limpio = re.sub(r"\s{2,}", " ", titulo_limpio).strip()

    if not bool(row.get("es_original", False)):
        # Es genérico
        titulo_limpio = f"{titulo_limpio} - Generico"
    else:
        titulo_limpio = f"{titulo_limpio} - Original"
    
    # 2. Marca efectiva (puede traer varias marcas separadas por |)
    marca = str(row.get("__marca_efectiva", row.get("marca", ""))).strip()
    
    # 3. Modelos ya consolidados
    modelo = row.get("__modelo_str", "")
    
    # Construcción final: título limpio, marca, modelos (si existen)
    if titulo_limpio:
        partes.append(f"{titulo_limpio}")
    if marca:
        partes.append(f"Marca: {marca}")
    if modelo:
        partes.append(f"Modelos: {modelo}")
    
    return " \n".join(partes)

def construir_dataset_final_es(df_consolidado: pd.DataFrame) -> pd.DataFrame:
    df = df_consolidado.copy()

    # Derivados
    df["__marca_efectiva"] = df.apply(_marca_efectiva, axis=1)
    df["__marcas_lista"]   = df["__marca_efectiva"].apply(_lista_marcas)
    df["__modelos_lista"]  = df.apply(_lista_modelos, axis=1)
    df["__modelo_str"]     = df["__modelos_lista"].apply(_modelo_str)
    df["__tipo_repuesto"]  = df.apply(_tipo_repuesto, axis=1)
    df["__etiquetas"]      = df["tags"].apply(_a_lista_desde_tags)
    df["__dimensiones"]    = df.apply(_dicc_dimensiones, axis=1)

    # Texto para embeddings
    df["texto"] = df.apply(_texto_para_embedding, axis=1)

    # Selección y renombre a español (conservando id_producto)
    df_final = pd.DataFrame({
        "id_producto": df["id_producto"].astype(str),
        "titulo": df["titulo_producto"].astype(str),
        "descripcion": df["descripcion"].astype(str),
        "texto": df["texto"],                          # campo a vectorizar
        "url": df["url_tienda"].astype(str),

        "marca_original": df["marca"].astype(str),     # puede ser GENERICO
        "marca": df["__marca_efectiva"].astype(str),   # marca efectiva (puede ser 'A | B' si generico)
        "marcas_lista": df["__marcas_lista"],          # lista (para filtros IN)

        "modelos_lista": df["__modelos_lista"],        # todas las refs encontradas
        "modelo": df["__modelo_str"],                  # join con |

        "categoria": df["categoria_general"].astype(str),  # categoría principal
        "subcategoria": df.get("subtipo_producto", ""),# opcional

        "tipo_repuesto": df["__tipo_repuesto"],        # ORIGINAL / GENERICO
        "es_llanta": df["es_llanta"],

        "dimensiones": df["__dimensiones"],            # dict (solo llantas)
        "dimensiones_str": df["dimensiones_str"],    # string humano

        "etiquetas": df["__etiquetas"],                # tags normalizados
        "precio": df["precio_producto"],                # tags normalizados
    })

    return df_final

In [ ]:
# df_consolidado = ... (tu dataframe tras consolidar modelos y dimensiones)
df_final = construir_dataset_final_es(df_consolidado)

# vistazo
df_final.head()

In [ ]:
df_final.iloc[0,:].to_dict()

In [ ]:
df_final[df_final.marca_original=="GENERICO"].iloc[4000,:].to_dict()

In [ ]:
df_final.sample(10).to_dict(orient="records")

In [ ]:
df_final.to_excel("../data/curated/shopify_data_to_index.xlsx", index=False)
#df_final.to_pickle("../data/curated/shopify_data_to_index.pkl")

In [ ]:
df_final.marca_original.value_counts().to_dict()

In [ ]:
df_final.marca_original.value_counts().to_dict()

In [ ]:
df_final.tipo_repuesto.value_counts().to_dict()

In [ ]:
df_final.categoria.value_counts().to_dict()

In [ ]:
df_final[df_final.categoria=='EMBRAGUE / CLUTCH'].sample(20).to_dict(orient="records")

In [ ]:
df_final[df_final.categoria=='TRANSMISION SECUNDARIA'].sample(10).to_dict(orient="records")

In [ ]:
df_final[df_final.categoria=='LLANTA'].sample(10).to_dict(orient="records")

In [ ]:
df_final.subcategoria.value_counts().to_dict()

In [ ]:
df_final.subcategoria.value_counts().to_dict()

In [ ]:
cols =  ["titulo"]
value_counts_dict = {" - ".join(cols): df_final[cols].value_counts().to_dict()}
value_counts_dict

In [ ]:
import re
import pandas as pd
from typing import Dict, List, Tuple, Optional, Any

# ====== helpers de normalización y regex (mismos que antes) ======

def canon_upper(s: str) -> str:
    s2 = s.upper()
    s2 = re.sub(r"[\s\-_\.]+", " ", s2)
    s2 = re.sub(r"\s+", " ", s2).strip()
    return s2

_SEP = r"[\s\-_\.]*"

def _split_letters_digits(token: str) -> List[str]:
    pieces = re.findall(r"[A-Z]+|\d+", token)
    return pieces if pieces else [token]

def _variant_to_pattern(variant: str) -> str:
    v = canon_upper(variant)
    tokens: List[str] = []
    for coarse in v.split():
        tokens += _split_letters_digits(coarse)
    parts = [re.escape(t) for t in tokens if t]
    core = _SEP.join(parts) if parts else re.escape(v)
    return rf"(?<![A-Z0-9]){core}(?![A-Z0-9])"

def build_index_from_consolidated(consolidated: Dict[str, Any]) -> List[Dict[str, Any]]:
    entries: List[Dict[str, Any]] = []
    for g in consolidated.get("grupos", []):
        ref_base = canon_upper(g["referencia"])
        variantes = g.get("variantes", []) or [ref_base]
        # asegúrate de incluir la referencia base como variante
        if ref_base not in (canon_upper(x) for x in variantes):
            variantes = [ref_base] + variantes
        for v in variantes:
            pat = _variant_to_pattern(v)
            tokens_count = len(re.findall(r"[A-Z]+|\d+", canon_upper(v)))
            weight = (len(canon_upper(v)), tokens_count)
            entries.append({
                "referencia_base": ref_base,
                "variante": v,
                "regex": re.compile(pat),
                "weight": weight,
            })
    # más largo / más tokens primero
    entries.sort(key=lambda e: e["weight"], reverse=True)
    return entries

def match_first(title: str, index: List[Dict[str, Any]]) -> Dict[str, Any]:
    if not isinstance(title, str) or not title:
        return {"ref_match": None, "variante_match": None, "match_span": None, "match_text": None}
    t = canon_upper(title)
    for ent in index:
        m = ent["regex"].search(t)
        if m:
            span = m.span()
            return {
                "ref_match": ent["referencia_base"],
                "variante_match": ent["variante"],
                "match_span": span,
                "match_text": t[span[0]:span[1]],
            }
    return {"ref_match": None, "variante_match": None, "match_span": None, "match_text": None}

# ====== función para DataFrame ======

def apply_reference_matching(
    df: pd.DataFrame,
    title_col: str,
    consolidated: Dict[str, Any],
) -> pd.DataFrame:
    """
    Añade columnas con el match de referencia a un DataFrame.
    - df: DataFrame de entrada
    - title_col: nombre de la columna con el título del repuesto
    - consolidated: JSON consolidado (marca, grupos, ...)
    """
    if title_col not in df.columns:
        raise ValueError(f"'{title_col}' no existe en el DataFrame")
    index = build_index_from_consolidated(consolidated)

    # aplica fila a fila (rápido para ~decenas/centenas de miles; si necesitas más, se puede paralelizar)
    matches = df[title_col].apply(lambda s: match_first(s, index))

    # expandir dict a columnas
    matches_df = pd.DataFrame(list(matches.values)) if hasattr(matches, "values") else pd.DataFrame(matches.tolist())
    out = df.reset_index(drop=True).join(matches_df)
    return out

In [ ]:
from pathlib import Path
from typing import Any, Dict

def load_json(path: str | Path) -> Dict[str, Any]:
    """
    Carga un archivo JSON desde la ruta dada y devuelve un dict de Python.
    
    Parámetros:
        path (str | Path): Ruta al archivo JSON.

    Devuelve:
        dict: Contenido del JSON como diccionario.
    """
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"No se encontró el archivo: {p.resolve()}")
    with open(p, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

In [ ]:
from glob import glob

files = glob("../data/results/curated/consolidacion_referencias/*.json")

for file in files:
    with open(file, "r") as f:
        data = json.load(f)
    print(data)

In [ ]:
RESULTS_DIR = "../data/results/curated/consolidacion_referencias"
DATA_PATH = "../data/curated/shopify_data_cat_gen.pkl"
TITLE_COL = "titulo_producto"

df = pd.read_pickle(DATA_PATH)  # o pd.read_csv(...)
# Asegúrate que la columna de marca exista:
assert "marca" in df.columns, "La columna 'marca' no existe en el dataset base"

# --- Iterar JSONs por marca, filtrar y consolidar ---
files = sorted(glob(f"{RESULTS_DIR}/*.json"))
dfs_out: List[pd.DataFrame] = []
row_count_expected = 0

for fp in files:
    consolidated_data = load_json(fp)
    marca_json = str(consolidated_data.get("marca", "")).upper()
    if not marca_json:
        # fallback: inferir desde el nombre del archivo (AKT_YYYYMMDD_*.json o AKT.json)
        stem = Path(fp).stem
        marca_json = stem.split("_")[0].upper()

    # filtrar dataset por marca
    df_marca = df[df["marca"].str.upper() == marca_json].copy()
    if df_marca.empty:
        print(f"[AVISO] No hay filas para marca '{marca_json}' en el dataset base. Archivo: {fp}")
        continue

    row_count_expected += len(df_marca)

    # aplicar matching referencias a esta marca
    df_matched = apply_reference_matching(df_marca, TITLE_COL, consolidated_data)
    # (opcional) guarda la marca normalizada, por si no lo está
    df_matched["marca_norm"] = marca_json

    dfs_out.append(df_matched)
    print(f"✔ {marca_json}: {len(df_marca)} filas procesadas")

# --- DataFrame consolidado ---
if dfs_out:
    df_consolidado = pd.concat(dfs_out, ignore_index=True)
else:
    df_consolidado = pd.DataFrame()

print("\nResumen")
print("  Archivos procesados:", len(files))
print("  Filas esperadas (suma por marca):", row_count_expected)
print("  Filas en df_consolidado:", len(df_consolidado))

# Verificación dura: deben ser iguales
assert len(df_consolidado) == row_count_expected, \
    "El consolidado no tiene el mismo número de filas que la suma de las marcas analizadas."

In [ ]:
df_consolidado[df_consolidado['ref_match'].notna()]

In [ ]:
df_consolidado.marca.value_counts().shape

In [ ]:
df.marca.value_counts()

In [ ]:
marcas_llantas = ["MICHELIN", "PIRELLI", "BENELLI", "KONTROL", "METZELER"] # para llantas el modelo  (ref_match) no aplica  

# para "GENERICO" toca aplicar la logica de ref pero con todos los json consolidados (se tendria una columna de ref_match por cada marca) luego se consolida con todas las referencias diferentes que hayan macheado.. Tambien se debe hererdad la marca del producto con el que macheo para completar estta columna para la marca GENERICO

In [ ]:
# Si el archivo está en data/results/consolidacion_referencias/AKT_20250826_101500.json
json_path = "../data/results/curated/consolidacion_referencias/AKT.json"
consolidated_data = load_json(json_path)

print(consolidated_data["marca"])       # ejemplo: AKT
print(len(consolidated_data["grupos"])) 


marca = "AKT"
df = pd.read_pickle("../data/curated/shopify_data_cat_gen.pkl")
df_marca = df[df.marca==marca].reset_index(drop=True)
df_marca.head()

In [ ]:
df_marca.shape

In [ ]:
df_out = apply_reference_matching(df_marca, title_col="titulo_producto", consolidated=consolidated_data)

# opcional: filtrar solo filas con match
df_matched = df_out[df_out["ref_match"].notna()]

# guardar resultados
df_out.to_csv("matched_referencias.csv", index=False)

In [ ]:
df_out.shape, df_matched.shape

In [ ]:
df_out[df_out['ref_match'].notna()].head(3)

In [ ]:
df_out[df_out['ref_match'].isna()].head(3)

In [ ]:
df_out[df_out['ref_match'].notna()].categoria_general.value_counts()

In [ ]:
df_out['tipo_producto'].value_counts()